In [ ]:
!pip install tensorflow
!pip install keras-tuner

In [5]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from keras import layers
from keras import Sequential
from sklearn.model_selection import train_test_split, GridSearchCV
from kerastuner.tuners import RandomSearch
from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier

In [6]:
df = pd.read_csv("/content/post_one_hot_encode.csv")

In [ ]:
train, test = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

x_train=train.loc[:, ['Power', 'Fin Type_Round', 'Fin Type_Square', 'Fin Type_Triangular', 'Configuration_Inline', 'Configuration_Staggered', 'PCM_Paraffin', 'PCM_RT-35', 'PCM_RT-44', 'Temperature']]
x_train=x_train.to_numpy()
y_train=train.loc[:, 'Time']
y_train=y_train.to_numpy()
x_test=test.loc[:, ['Power', 'Fin Type_Round', 'Fin Type_Square', 'Fin Type_Triangular', 'Configuration_Inline', 'Configuration_Staggered', 'PCM_Paraffin', 'PCM_RT-35', 'PCM_RT-44', 'Temperature']]
x_test=x_test.to_numpy()
y_test=test.loc[:, 'Time']
y_test=y_test.to_numpy()

In [9]:
#Build model function
def build_model(activation = 'relu', optimizer = 'adam'):
    model = keras.Sequential()
    model.add(layers.Dense(40, activation=activation))
    model.add(layers.Dense(50, activation=activation))
    model.add(layers.Dense(50, activation=activation))
    model.add(layers.Dense(35, activation=activation))
    model.add(layers.Dense(30, activation=activation))
    model.add(layers.Dense(45, activation=activation))
    model.add(layers.Dense(15, activation=activation))
    model.add(layers.Dense(35, activation=activation))

    model.add(layers.Dense(1, activation='linear'))
    model.compile(loss="mean_absolute_error", optimizer=optimizer)
    return model

In [10]:
#creating the model with KerasRegressor
model = KerasRegressor(
    build_fn = build_model,
    epochs = 10,
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  after removing the cwd from sys.path.


In [11]:
activation = ['relu', 'softmax', 'tanh', 'sigmoid']
optimizer = ['Adam', 'SGD', 'RMSprop']

In [12]:
param_grid=dict(activation=activation, optimizer=optimizer)

In [13]:
grid = GridSearchCV(estimator=model, param_grid=param_grid)

In [14]:
grid_result = grid.fit(x_train, y_train)

Epoch 1/10
34/34 [==============================] - 1s 3ms/step - loss: 33.8313
Epoch 2/10
34/34 [==============================] - 0s 3ms/step - loss: 18.0391
Epoch 3/10
34/34 [==============================] - 0s 2ms/step - loss: 16.6833
Epoch 4/10
34/34 [==============================] - 0s 4ms/step - loss: 16.4030
Epoch 5/10
34/34 [==============================] - 0s 3ms/step - loss: 15.8878
Epoch 6/10
34/34 [==============================] - 0s 3ms/step - loss: 15.4635
Epoch 7/10
34/34 [==============================] - 0s 2ms/step - loss: 13.8925
Epoch 8/10
34/34 [==============================] - 0s 2ms/step - loss: 11.1895
Epoch 9/10
34/34 [==============================] - 0s 3ms/step - loss: 10.6751
Epoch 10/10
9/9 [==============================] - 1s 3ms/step - loss: 9.4400
Epoch 1/10
34/34 [==============================] - 1s 5ms/step - loss: 31.5798
Epoch 2/10
34/34 [==============================] - 0s 6ms/step - loss: 17.0967
Epoch 3/10
34/34 [========================

In [15]:
#The score is negeted, as GridSearchCV tries to maximize the score
print("Best: %f using %s" % (-grid_result.best_score_, grid_result.best_params_))
means = -grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean = %f (std=%f) with: %r" % (mean, stdev, param))

Best: 10.736630 using {'activation': 'relu', 'optimizer': 'Adam'}
Mean = 10.736630 (std=1.078784) with: {'activation': 'relu', 'optimizer': 'Adam'}
Mean = 34.561745 (std=1.941455) with: {'activation': 'relu', 'optimizer': 'SGD'}
Mean = 18.121083 (std=2.355369) with: {'activation': 'relu', 'optimizer': 'RMSprop'}
Mean = 43.895967 (std=1.475078) with: {'activation': 'softmax', 'optimizer': 'Adam'}
Mean = 41.585633 (std=1.496371) with: {'activation': 'softmax', 'optimizer': 'SGD'}
Mean = 43.916382 (std=1.482439) with: {'activation': 'softmax', 'optimizer': 'RMSprop'}
Mean = 30.247390 (std=1.624947) with: {'activation': 'tanh', 'optimizer': 'Adam'}
Mean = 23.375412 (std=0.851874) with: {'activation': 'tanh', 'optimizer': 'SGD'}
Mean = 29.901046 (std=1.231997) with: {'activation': 'tanh', 'optimizer': 'RMSprop'}
Mean = 33.849682 (std=2.005357) with: {'activation': 'sigmoid', 'optimizer': 'Adam'}
Mean = 24.055581 (std=1.039336) with: {'activation': 'sigmoid', 'optimizer': 'SGD'}
Mean = 35.17